## Desafio Banco do Brasil

In [1]:
# Imports Pandas e Numpy
import pandas as pd
import numpy as np

# Import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Read csv
base = pd.read_csv('census.csv',delimiter=';')

In [3]:
# Mostra os 5 primeiros registros do dataset
base.head(5)  

,age,workclass,education_level,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,Bachelors,13.0,Never-married,Adm-clerical,Not-in-family,White,Male,2174.0,0.0,40.0,United-States,<=50K
1,50,Self-emp-not-inc,Bachelors,13.0,Married-civ-spouse,Exec-managerial,Husband,White,Male,0.0,0.0,13.0,United-States,<=50K
2,38,Private,HS-grad,9.0,Divorced,Handlers-cleaners,Not-in-family,White,Male,0.0,0.0,40.0,United-States,<=50K
3,53,Private,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
4,28,Private,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0.0,0.0,40.0,Cuba,<=50K


In [4]:
# Verifica a quantidade de nulos 
base.isnull().sum()

age                0
workclass          0
education_level    0
education-num      0
marital-status     0
occupation         0
relationship       0
race               0
sex                0
capital-gain       1
capital-loss       1
hours-per-week     1
native-country     0
income             1
dtype: int64

In [5]:
# Info para verificacao de tipos de colunas, quantidade e missings
base.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5533 entries, 0 to 5532
Data columns (total 14 columns):
age                5533 non-null int64
workclass          5533 non-null object
education_level    5533 non-null object
education-num      5533 non-null float64
marital-status     5533 non-null object
occupation         5533 non-null object
relationship       5533 non-null object
race               5533 non-null object
sex                5533 non-null object
capital-gain       5532 non-null float64
capital-loss       5532 non-null float64
hours-per-week     5532 non-null float64
native-country     5533 non-null object
income             5532 non-null object
dtypes: float64(4), int64(1), object(9)
memory usage: 605.2+ KB


In [6]:
# Coloca a media nos missing capital loss
filtro = base['capital-loss'].isnull()
base.loc[filtro,'capital-loss'] = base['capital-loss'].mean()

In [7]:
# cria os previsores e a classe
previsores = base.iloc[:, 0:13].values
classe = base.iloc[:, -1].values

In [8]:
# transforma as variaveis para inteiros com o LabelEncode dos previsores
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_previsores = LabelEncoder()

previsores[:, 1] = labelencoder_previsores.fit_transform(previsores[:, 1])
previsores[:, 2] = labelencoder_previsores.fit_transform(previsores[:, 2])
previsores[:, 3] = labelencoder_previsores.fit_transform(previsores[:, 3])
previsores[:, 4] = labelencoder_previsores.fit_transform(previsores[:, 4])
previsores[:, 5] = labelencoder_previsores.fit_transform(previsores[:, 5])
previsores[:, 6] = labelencoder_previsores.fit_transform(previsores[:, 6])
previsores[:, 7] = labelencoder_previsores.fit_transform(previsores[:, 7])
previsores[:, 8] = labelencoder_previsores.fit_transform(previsores[:, 8])
previsores[:, 9] = labelencoder_previsores.fit_transform(previsores[:, 9])
previsores[:, 11] = labelencoder_previsores.fit_transform(previsores[:, 11])
previsores[:, 12] = labelencoder_previsores.fit_transform(previsores[:, 12])

In [9]:
# OneHotEncoder classifica as colunas
onehotencoder = OneHotEncoder(categorical_features = [1,2,3,4,5,6,7,8,9,11,12])
previsores = onehotencoder.fit_transform(previsores).toarray()

In [10]:
# transforma as variaveis para inteiros com o LabelEncode da classe (target)
labelencoder_classe = LabelEncoder()
classe = labelencoder_classe.fit_transform(classe)

In [11]:
# gera a curva de classificacao
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
previsores = scaler.fit_transform(previsores)

In [12]:
# realiza o treinamento com 30% dos dados 
from sklearn.cross_validation import train_test_split
previsores_treinamento, previsores_teste, classe_treinamento, classe_teste = train_test_split(previsores, classe, test_size=0.30, random_state=0)

/anaconda3/envs/python2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [13]:
# Gaussian do treinamento para verificar se ele gera uma acuracia melhor
from sklearn.naive_bayes import GaussianNB
classificador = GaussianNB()
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

In [14]:
# realiza a matriz de confusao e gera a acuracia
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

In [15]:
# imprimi a precisao de acuracia e verifica que nao foi bom desempenho
print(precisao)

0.32349397590361445


In [16]:
# classificacao de arvore de decisao para verificar se resulta melhor acuracia 
from sklearn.ensemble import RandomForestClassifier
classificador = RandomForestClassifier(n_estimators=40, criterion='entropy', random_state=0)
classificador.fit(previsores_treinamento, classe_treinamento)
previsoes = classificador.predict(previsores_teste)

In [17]:
# realiza a matriz de confusao e gera a acuracia
from sklearn.metrics import confusion_matrix, accuracy_score
precisao = accuracy_score(classe_teste, previsoes)
matriz = confusion_matrix(classe_teste, previsoes)

In [18]:
# imprimi o resultado e verifica melhor acuracia
print(precisao)

0.8307228915662651


In [19]:
# separa os maiores que 50 para gerar a probabilidade
maiores_50 = base[base.income==">50K"].income.count()
num_reg    = base.count()

probabilidade = maiores_50/num_reg

In [20]:
# Calcula o log loss
perda_precisao = 1e-14
predicao = np.clip(precisao, perda_precisao, 1-perda_precisao)
loss = -1 * np.mean(probabilidade * np.log(predicao) + (1-probabilidade) * np.log(1-predicao))


In [21]:
# imprimi o log loss
print(loss)

1.37570973329
